## EXP125,126,127
### category123のfeatureをエンコーディングして特徴量にする

In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [39]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_hist = utils.read_df_pkl('../input/hist*0*')
df_new = utils.read_df_pkl('../input/new*0*')

100%|██████████| 3/3 [00:00<00:00, 106.20it/s]


SystemExit: 

In [3]:
for df in [df_hist,df_new]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['authorized_flag'] = df['authorized_flag'].map(lambda x:0 if x=='N' else 1)

### 125 category別にauth_meanをとりワンホットで特徴量にする

In [38]:
def get_new_columns(name, aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

aggs = {}
aggs['authorized_flag'] = ['sum', 'mean']

category_cols = [col for col in df_hist.columns if col.count('category_')]

agg_list = []
cat1 = df_hist.groupby([key, 'category_1'])['authorized_flag'].agg(aggs)
cat2 = df_hist.groupby([key, 'category_2'])['authorized_flag'].agg(aggs)
cat3 = df_hist.groupby([key, 'category_3'])['authorized_flag'].agg(aggs)

new_columns = get_new_columns('hist', aggs)
for df in [cat1, cat2, cat3]:
    df.columns = new_columns

In [43]:
# cat1
cat1 = cat1.unstack()
cat1_list = []
for col in cat1.columns:
    tmp = cat1[col]
    tmp.name = 'category1_' + col[1] + '_' + col[0]
    cat1_list.append(tmp)
df_cat1 = pd.concat(cat1_list, axis=1)

# cat2
cat2 = cat2.unstack()
cat2_list = []
for col in cat2.columns:
    tmp = cat2[col]
    tmp.name = 'category2_' + str(int(col[1])) + '_' + col[0]
    cat2_list.append(tmp)
df_cat2 = pd.concat(cat2_list, axis=1)

# cat3
cat3 = cat3.unstack()
cat3_list = []
for col in cat3.columns:
    tmp = cat3[col]
    tmp.name = 'category3_' + col[1] + '_' + col[0]
    cat3_list.append(tmp)
df_cat3 = pd.concat(cat3_list, axis=1)

df_train = df_train.set_index(key).join(df_cat1)
df_train = df_train.join(df_cat2)
df_train = df_train.join(df_cat3)

df_test = df_test.set_index(key).join(df_cat1)
df_test = df_test.join(df_cat2)
df_test = df_test.join(df_cat3)

df_train.head()

,first_active_month,feature_1,feature_2,feature_3,target,category1_N_hist_authorized_flag_sum,category1_Y_hist_authorized_flag_sum,category1_N_hist_authorized_flag_mean,category1_Y_hist_authorized_flag_mean,category2_1_hist_authorized_flag_sum,category2_2_hist_authorized_flag_sum,category2_3_hist_authorized_flag_sum,category2_4_hist_authorized_flag_sum,category2_5_hist_authorized_flag_sum,category2_1_hist_authorized_flag_mean,category2_2_hist_authorized_flag_mean,category2_3_hist_authorized_flag_mean,category2_4_hist_authorized_flag_mean,category2_5_hist_authorized_flag_mean,category3_A_hist_authorized_flag_sum,category3_B_hist_authorized_flag_sum,category3_C_hist_authorized_flag_sum,category3_A_hist_authorized_flag_mean,category3_B_hist_authorized_flag_mean,category3_C_hist_authorized_flag_mean
card_id,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,247.0,NaN,0.950000,NaN,244.0,NaN,NaN,NaN,3.0,0.949416,NaN,NaN,NaN,1.000000,247.0,0.0,NaN,0.964844,0.000000,NaN
C_ID_3d0044924f,2017-01,4,1,0,0.392913,310.0,29.0,0.971787,0.935484,310.0,NaN,NaN,NaN,NaN,0.971787,NaN,NaN,NaN,NaN,NaN,272.0,65.0,NaN,0.985507,0.902778
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,41.0,NaN,0.953488,NaN,4.0,NaN,NaN,NaN,37.0,1.000000,NaN,NaN,NaN,0.948718,41.0,NaN,NaN,0.953488,NaN,NaN
C_ID_186d6a6901,2017-09,4,3,0,0.142495,65.0,12.0,1.000000,1.000000,12.0,NaN,NaN,53.0,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,68.0,7.0,NaN,1.000000,1.000000
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,116.0,12.0,0.983051,0.800000,10.0,NaN,NaN,105.0,1.0,1.000000,NaN,NaN,0.981308,1.000000,NaN,124.0,4.0,NaN,0.984127,0.571429


In [44]:
feat_no = '125_cat_'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if (col.count('feature')):continue
    if col in ignore_features: continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)

### 126 auth1, 0でのcategory123のfreq

In [45]:
df_hist['cnt'] = 1
df_new['cnt'] = 1
auth1_hist = df_hist[df_hist.authorized_flag==1]
auth0_hist = df_hist[df_hist.authorized_flag==0]

In [73]:
df = auth1_hist

cat1 = df.groupby([key, 'category_1'])['cnt'].sum()
cat2 = df.groupby([key, 'category_2'])['cnt'].sum()
cat3 = df.groupby([key, 'category_3'])['cnt'].sum()

cat1.name = 'auth1_hist_category_1'
cat2.name = 'auth1_hist_category_2'
cat3.name = 'auth1_hist_category_3'

# cat1
df_tmp = cat1.unstack()
cat1_list = []
for col in df_tmp.columns:
    tmp = df_tmp[col]
    tmp.name = cat1.name + '_' + col + '_cnt'
    cat1_list.append(tmp)
df_cat1 = pd.concat(cat1_list, axis=1)

# cat2
df_tmp = cat2.unstack()
cat2_list = []
for col in df_tmp.columns:
    tmp = df_tmp[col]
    tmp.name = cat2.name + '_' + str(int(col)) + '_cnt'
    cat2_list.append(tmp)
df_cat2 = pd.concat(cat2_list, axis=1)

# cat3
df_tmp = cat3.unstack()
cat3_list = []
for col in df_tmp.columns:
    tmp = df_tmp[col]
    tmp.name = cat3.name + '_' + col + '_cnt'
    cat3_list.append(tmp)
df_cat3 = pd.concat(cat3_list, axis=1)


df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')

df_train = df_train.set_index(key).join(df_cat1)
df_train = df_train.join(df_cat2)
df_train = df_train.join(df_cat3)

df_test = df_test.set_index(key).join(df_cat1)
df_test = df_test.join(df_cat2)
df_test = df_test.join(df_cat3)

df_train.head()

100%|██████████| 3/3 [00:00<00:00, 164.92it/s]


,first_active_month,feature_1,feature_2,feature_3,target,auth1_hist_category_1_N_cnt,auth1_hist_category_1_Y_cnt,auth1_hist_category_2_1_cnt,auth1_hist_category_2_2_cnt,auth1_hist_category_2_3_cnt,auth1_hist_category_2_4_cnt,auth1_hist_category_2_5_cnt,auth1_hist_category_3_A_cnt,auth1_hist_category_3_B_cnt,auth1_hist_category_3_C_cnt
card_id,,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,247.0,NaN,244.0,NaN,NaN,NaN,3.0,247.0,NaN,NaN
C_ID_3d0044924f,2017-01,4,1,0,0.392913,310.0,29.0,310.0,NaN,NaN,NaN,NaN,NaN,272.0,65.0
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,41.0,NaN,4.0,NaN,NaN,NaN,37.0,41.0,NaN,NaN
C_ID_186d6a6901,2017-09,4,3,0,0.142495,65.0,12.0,12.0,NaN,NaN,53.0,NaN,NaN,68.0,7.0
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,116.0,12.0,10.0,NaN,NaN,105.0,1.0,NaN,124.0,4.0


In [74]:
feat_no = '126_cat_'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if (col.count('feature')):continue
    if col in ignore_features: continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)

### 127 auth0でのcategory123のfreq

In [75]:
df = auth0_hist

cat1 = df.groupby([key, 'category_1'])['cnt'].sum()
cat2 = df.groupby([key, 'category_2'])['cnt'].sum()
cat3 = df.groupby([key, 'category_3'])['cnt'].sum()

cat1.name = 'auth0_hist_category_1'
cat2.name = 'auth0_hist_category_2'
cat3.name = 'auth0_hist_category_3'

# cat1
df_tmp = cat1.unstack()
cat1_list = []
for col in df_tmp.columns:
    tmp = df_tmp[col]
    tmp.name = cat1.name + '_' + col + '_cnt'
    cat1_list.append(tmp)
df_cat1 = pd.concat(cat1_list, axis=1)

# cat2
df_tmp = cat2.unstack()
cat2_list = []
for col in df_tmp.columns:
    tmp = df_tmp[col]
    tmp.name = cat2.name + '_' + str(int(col)) + '_cnt'
    cat2_list.append(tmp)
df_cat2 = pd.concat(cat2_list, axis=1)

# cat3
df_tmp = cat3.unstack()
cat3_list = []
for col in df_tmp.columns:
    tmp = df_tmp[col]
    tmp.name = cat3.name + '_' + col + '_cnt'
    cat3_list.append(tmp)
df_cat3 = pd.concat(cat3_list, axis=1)


df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')

df_train = df_train.set_index(key).join(df_cat1)
df_train = df_train.join(df_cat2)
df_train = df_train.join(df_cat3)

df_test = df_test.set_index(key).join(df_cat1)
df_test = df_test.join(df_cat2)
df_test = df_test.join(df_cat3)

df_train.head()

100%|██████████| 3/3 [00:00<00:00, 76.04it/s]


,first_active_month,feature_1,feature_2,feature_3,target,auth0_hist_category_1_N_cnt,auth0_hist_category_1_Y_cnt,auth0_hist_category_2_1_cnt,auth0_hist_category_2_2_cnt,auth0_hist_category_2_3_cnt,auth0_hist_category_2_4_cnt,auth0_hist_category_2_5_cnt,auth0_hist_category_3_A_cnt,auth0_hist_category_3_B_cnt,auth0_hist_category_3_C_cnt
card_id,,,,,,,,,,,,,,,
C_ID_92a2005557,2017-06,5,2,1,-0.820283,13.0,NaN,13.0,NaN,NaN,NaN,NaN,9.0,4.0,NaN
C_ID_3d0044924f,2017-01,4,1,0,0.392913,9.0,2.0,9.0,NaN,NaN,NaN,NaN,NaN,4.0,7.0
C_ID_d639edf6cd,2016-08,2,2,0,0.688056,2.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN
C_ID_186d6a6901,2017-09,4,3,0,0.142495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159749,2.0,3.0,NaN,NaN,NaN,2.0,NaN,NaN,2.0,3.0


In [76]:
feat_no = '127_cat_'
ignore_features = ['first_active_month', 'card_id', target]

for col in df_train.columns:
    if (col.count('feature')):continue
    if col in ignore_features: continue
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{feat_no}test_{col}@', obj=df_test[col].values)

array([False, False, False, False, False, False, False, False, False])

### 124 same date feature